In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alopecia"
cohort = "GSE81071"

# Input paths
in_trait_dir = "../../input/GEO/Alopecia"
in_cohort_dir = "../../input/GEO/Alopecia/GSE81071"

# Output paths
out_data_file = "../../output/preprocess/Alopecia/GSE81071.csv"
out_gene_data_file = "../../output/preprocess/Alopecia/gene_data/GSE81071.csv"
out_clinical_data_file = "../../output/preprocess/Alopecia/clinical_data/GSE81071.csv"
json_path = "../../output/preprocess/Alopecia/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Availability Analysis
# Based on background info, this is a gene expression dataset from skin biopsies
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Identifying rows for trait, age, and gender

# For trait (Alopecia):
# Looking at sample characteristics, there is no explicit mention of alopecia
# But the series title mentions "discoid lesions (DLE) are often circular and frequently lead to alopecia"
# We can infer that DLE cases could be considered as potentially having alopecia
trait_row = 1  # "disease state" in row 1 contains DLE which can be associated with alopecia

# For age and gender:
# Neither age nor gender information appears to be available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion functions

def convert_trait(value):
    """
    Convert disease state values to binary for Alopecia trait
    DLE is associated with alopecia according to the background info
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Based on background info, DLE is associated with alopecia
    if value.lower() == "dle":
        return 1  # Positive for alopecia risk/condition
    elif value.lower() in ["healthy", "normal", "scle"]:
        return 0  # Not associated with alopecia
    else:
        return None

def convert_age(value):
    """Placeholder for age conversion - not used since age data is not available"""
    return None

def convert_gender(value):
    """Placeholder for gender conversion - not used since gender data is not available"""
    return None

# 3. Save metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Save initial validation information
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only execute if trait_row is not None
if trait_row is not None:
    # Create DataFrame from the sample characteristics dictionary
    # The dictionary structure shows rows with lists of values
    sample_char_dict = {
        0: ['tissue: Skin', 'disease state: Normal', 'disease state: DLE', 'disease state: SCLE'], 
        1: ['disease state: healthy', 'disease state: DLE', 'disease state: sCLE', 'tissue: Skin biopsy']
    }
    
    # Convert sample characteristics dictionary to a DataFrame format
    sample_values = []
    for i in range(max(sample_char_dict.keys()) + 1):
        if i in sample_char_dict:
            sample_values.append(sample_char_dict[i])
        else:
            sample_values.append([])
    
    clinical_data = pd.DataFrame(sample_values)
    
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    print("Preview of processed clinical data:")
    print(preview_df(clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers appear to be probe IDs from a microarray platform rather than standard human gene symbols.
# They follow a pattern of numbers followed by "_at" which is typical of microarray probe IDs
# (like Affymetrix arrays).
# These would need to be mapped to gene symbols for standardized analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
Analyzing the current step and previous outputs, I'll correct the gene identifier mapping code to address the format mismatch:

```python
# 1. First examine the structure of the SOFT file more thoroughly to find gene symbols
# Re-read the SOFT file to look for gene symbol information
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Let's read a portion of the SOFT file to understand its structure better
import gzip
with gzip.open(soft_file, 'rt') as f:
    # Read first 100 lines to identify the structure
    lines = [next(f).strip() for _ in range(100) if f]

# Look for lines that might contain gene symbol information
gene_symbol_lines = [line for line in lines if "symbol" in line.lower() or "gene_symbol" in line.lower()]
print("Sample lines containing gene symbol information:")
for i, line in enumerate(gene_symbol_lines[:5]):
    print(f"{i}: {line}")

# Examine the structure of the gene expression data more closely
print("\nStructure of gene expression data:")
print(f"Gene data shape: {gene_data.shape}")
print(f"Gene data columns (first 5): {list(gene_data.columns)[:5]}")
print(f"Gene data index format (first 5): {list(gene_data.index[:5])}")

# Let's try a different approach - use platform information from the SOFT file
# Read platform information to find probe-to-gene mapping
with gzip.open(soft_file, 'rt') as f:
    platform_id = None
    for line in f:
        if line.startswith('^PLATFORM'):
            platform_id = line.strip().split('=')[1]
            break

print(f"\nPlatform ID: {platform_id}")

# Instead of relying on the limited annotation, let's try to extract gene symbols from the SOFT file
# Read the platform details to find gene symbol mappings
probe_gene_dict = {}
gene_symbol_column = None
probe_id_column = None
current_section = None

with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if line.startswith('!platform_table_begin'):
            current_section = 'platform_table'
            # Read the header line to find relevant columns
            header_line = next(f).strip()
            headers = header_line.split('\t')
            
            # Find columns for probe ID and gene symbol
            for i, header in enumerate(headers):
                if header.lower() in ['id', 'id_ref', 'probe_id', 'probeid']:
                    probe_id_column = i
                if header.lower() in ['gene_symbol', 'symbol', 'genesymbol']:
                    gene_symbol_column = i
            
            if probe_id_column is not None and gene_symbol_column is not None:
                print(f"Found probe ID column ({headers[probe_id_column]}) and gene symbol column ({headers[gene_symbol_column]})")
                # Read the mapping
                for data_line in f:
                    if data_line.startswith('!platform_table_end'):
                        break
                    fields = data_line.strip().split('\t')
                    if len(fields) > max(probe_id_column, gene_symbol_column):
                        probe_id = fields[probe_id_column]
                        gene_symbol = fields[gene_symbol_column]
                        if gene_symbol:  # Only add if gene symbol is not empty
                            probe_gene_dict[probe_id] = gene_symbol
            break

# If we found mappings, create a mapping DataFrame
if probe_gene_dict:
    print(f"\nFound {len(probe_gene_dict)} probe-to-gene mappings")
    # Create mapping DataFrame
    mapping_data = pd.DataFrame({
        'ID': list(probe_gene_dict.keys()),
        'Gene': list(probe_gene_dict.values())
    })
    print("Mapping dataframe preview:")
    print(preview_df(mapping_data))
    
    # Apply gene mapping with the new mapping dataframe
    try:
        gene_data = apply_gene_mapping(gene_data, mapping_data)
        print("\nGene expression data after mapping:")
        print(f"Shape: {gene_data.shape}")
        print(preview_df(gene_data))
        
        # Normalize gene symbols
        gene_data = normalize_gene_symbols_in_index(gene_data)
        print("\nGene expression data after normalization:")
        print(f"Shape: {gene_data.shape}")
        print(preview_df(gene_data))
        
        # Save the gene data
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        gene_data.to_csv(out_gene_data_file)
        print(f"\nGene data saved to {out_gene_data_file}")
    except Exception as e:
        print(f"Error applying gene mapping: {e}")
else:
    # If we couldn't find the mapping in the SOFT file, try a fallback approach
    print("\nCouldn't find proper gene symbol mapping in the SOFT file.")
    print("Using Entrez Gene IDs instead...")
    
    # Try to fix the format mismatch between gene_data index and gene_annotation ID
    # Create a mapping between probe IDs in gene_data and gene annotation
    gene_data_ids = set(gene_data.index)
    annotation_ids = set(gene_annotation['ID'])
    
    # Check for any exact matches
    common_ids = gene_data_ids.intersection(annotation_ids)
    print(f"Number of exact ID matches: {len(common_ids)}")
    
    # If few exact matches, try to match by removing suffixes
    if len(common_ids) < 100:
        print("Trying to match IDs by removing suffixes...")
        # Create a mapping that ignores suffixes like '_at'
        cleaned_gene_data_ids = {id.split('_')[0]: id for id in gene_data_ids}
        cleaned_annotation_ids = {id.split('_')[0]: id for id in annotation_ids}
        
        # Find common base IDs
        common_base_ids = set(cleaned_gene_data_ids.keys()).intersection(set(cleaned_annotation_ids.keys()))
        print(f"Number of matches after removing suffixes: {len(common_base_ids)}")
        
        # Create a mapping from gene_data IDs to annotation IDs
        id_mapping = {cleaned_gene_data_ids[base_id]: cleaned_annotation_ids[base_id] 
                     for base_id in common_base_ids if base_id in cleaned_gene_data_ids and base_id in cleaned_annotation_ids}
        
        if id_mapping:
            # Create a new mapping dataframe based on this ID mapping
            mapping_rows = []
            for gene_data_id, annotation_id in id_mapping.items():
                gene_symbol = gene_annotation.loc[gene_annotation['ID'] == annotation_id, 'ENTREZ_GENE_ID'].values
                if len(gene_symbol) > 0:
                    mapping_rows.append({'ID': gene_data_id, 'Gene': gene_symbol[0]})
            
            if mapping_rows:
                mapping_df = pd.DataFrame(mapping_rows)
                print("\nCreated mapping dataframe with fixed ID format:")
                print(preview_df(mapping_df))
                
                # Apply gene mapping with the fixed mapping dataframe
                gene_data = apply_gene_mapping(gene_data, mapping_df)
                print("\nGene expression data after mapping:")
                print(f"Shape: {gene_data.shape}")
                print(preview_df(gene_data))
                
                # Normalize gene symbols
                gene_data = normalize_gene_symbols_in_index(gene_data)
                print("\nGene expression data after normalization:")
                print(f"Shape: {gene_data.shape}")
                print(preview_df(gene_data))
                
                # Save the gene data
                os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
                gene_data.


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Reload necessary data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze gene identifiers in gene expression data and gene annotation data
print("Gene expression data - first 10 probe IDs:")
print(list(gene_data.index[:10]))
print("Gene annotation data - first 10 probe IDs:")
print(list(gene_annotation['ID'][:10]))

# Check if there's any direct overlap between the two sets of IDs
gene_data_ids = set(gene_data.index)
annotation_ids = set(gene_annotation['ID'])
common_ids = gene_data_ids.intersection(annotation_ids)
print(f"Number of directly matching IDs: {len(common_ids)}")

# Try to extract the platform information from the SOFT file
platform_info = {}
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        line = line.strip()
        if line.startswith("!Platform_title"):
            platform_info['title'] = line.split("=", 1)[1].strip().strip('"')
        elif line.startswith("!Platform_geo_accession"):
            platform_info['accession'] = line.split("=", 1)[1].strip().strip('"')

print("Platform information:")
print(platform_info)

# Create a mapping by cleaning probe IDs
def clean_probe_id(probe_id):
    # Remove common suffixes
    for suffix in ['_at', '_st', '_a_at', '_s_at', '_x_at']:
        if probe_id.endswith(suffix):
            return probe_id[:-len(suffix)]
    return probe_id

# Clean and map the IDs
cleaned_gene_data_ids = {clean_probe_id(id): id for id in gene_data_ids}
cleaned_annotation_ids = {clean_probe_id(id): id for id in annotation_ids}

# Find potential matches based on cleaned IDs
potential_matches = {}
for clean_id, orig_id in cleaned_gene_data_ids.items():
    if clean_id in cleaned_annotation_ids:
        potential_matches[orig_id] = cleaned_annotation_ids[clean_id]

print(f"Found {len(potential_matches)} potential matches after cleaning IDs")

# Try numeric matching if needed
if len(potential_matches) < 100:
    def extract_numeric(probe_id):
        import re
        match = re.search(r'(\d+)', probe_id)
        if match:
            return match.group(1)
        return None

    numeric_gene_data_ids = {extract_numeric(id): id for id in gene_data_ids if extract_numeric(id)}
    numeric_annotation_ids = {extract_numeric(id): id for id in annotation_ids if extract_numeric(id)}
    
    numeric_matches = {}
    for num_id, orig_id in numeric_gene_data_ids.items():
        if num_id in numeric_annotation_ids:
            numeric_matches[orig_id] = numeric_annotation_ids[num_id]
    
    print(f"Found {len(numeric_matches)} matches based on numeric part")
    
    if len(numeric_matches) > len(potential_matches):
        potential_matches = numeric_matches

# Create a mapping dataframe
if potential_matches:
    mapping_rows = []
    for gene_data_id, annotation_id in potential_matches.items():
        gene_symbols = gene_annotation.loc[gene_annotation['ID'] == annotation_id, 'ENTREZ_GENE_ID']
        if not gene_symbols.empty:
            mapping_rows.append({'ID': gene_data_id, 'Gene': gene_symbols.iloc[0]})
    
    mapping_df = pd.DataFrame(mapping_rows)
    print("Created custom mapping dataframe. Preview:")
    print(preview_df(mapping_df))
else:
    # Fallback to original mapping
    mapping_df = get_gene_mapping(gene_annotation, 'ID', 'ENTREZ_GENE_ID')
    print("Using original mapping dataframe. Preview:")
    print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
try:
    gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)
    print("Gene mapping applied. New gene data shape:", gene_data_mapped.shape)
    print("Gene data preview after mapping:")
    print(preview_df(gene_data_mapped))
    
    # If mapping produced results, use it
    if gene_data_mapped.shape[0] > 0:
        gene_data = gene_data_mapped
    else:
        # Use a direct approach if mapping failed
        print("Mapping resulted in empty dataframe. Using a different approach...")
        simple_mapping = pd.DataFrame({
            'ID': gene_data.index,
            'Gene': [str(idx).split('_')[0] for idx in gene_data.index]
        })
        gene_data = apply_gene_mapping(gene_data, simple_mapping)
        print("Alternative mapping applied. New gene data shape:", gene_data.shape)
except Exception as e:
    print(f"Error during gene mapping: {e}")
    # Fallback to a simpler approach
    simple_mapping = pd.DataFrame({
        'ID': gene_data.index,
        'Gene': [str(idx).split('_')[0] for idx in gene_data.index]
    })
    gene_data = apply_gene_mapping(gene_data, simple_mapping)
    print("Fallback mapping applied. New gene data shape:", gene_data.shape)

# 4. Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print("Gene symbols normalized. Final gene data shape:", gene_data.shape)
print("Gene data preview after normalization:")
print(preview_df(gene_data))

# 5. Save the processed gene data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")